## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce

## Train data

In [2]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [3]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [4]:
train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


In [5]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [6]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [7]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()
    distances_df = sorted.apply(get_distances, axis=1)

    distances_df.columns = distances
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [8]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### One hot encoding

In [9]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [10]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [11]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [13]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.775959,1.163939,-0.775959,-0.775959,1.163939,0.537109,-1.596136,-1.329480,0.003798,0.270453,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.290985,-0.775959,0.678965,-0.775959,1.163939,-0.796169,-0.262858,1.337076,1.070420,0.003798,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.260934,1.648914,0.678965,-1.260934,0.193990,1.070420,-1.596136,0.003798,1.070420,-1.596136,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.290985,0.193990,-1.260934,-1.260934,2.618863,0.537109,-0.796169,-0.262858,0.537109,0.537109,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.775959,-0.290985,0.678965,0.193990,0.193990,0.270453,-0.796169,1.070420,-1.329480,-1.596136,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
data_normalized.shape

(25010, 30)

#### Test function

In [15]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [16]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Neural Network

In [37]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.1))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(60, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=300, batch_size=256)

Epoch 1/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 50us/step - loss: 1.1721 - acc: 0.5334

Epoch 2/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 26us/step - loss: 0.8731 - acc: 0.6383

Epoch 3/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.7421 - acc: 0.6990

Epoch 4/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.6079 - acc: 0.7643

Epoch 5/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 34us/step - loss: 0.4510 - acc: 0.8467

Epoch 6/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 28us/step - loss: 0.3143 - acc: 0.9061

Epoch 7/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 30us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 0.0057 - acc: 0.9984

Epoch 57/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 0.0051 - acc: 0.9988

Epoch 58/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 0.0038 - acc: 0.9989

Epoch 59/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0043 - acc: 0.9988

Epoch 60/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0049 - acc: 0.9986

Epoch 61/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0047 - acc: 0.9988

Epoch 62/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 0.0031

25010/25010 [==============================]25010/25010 [==============================] - 1s 26us/step - loss: 0.0012 - acc: 0.9996

Epoch 112/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 27us/step - loss: 7.7696e-04 - acc: 0.9998

Epoch 113/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0011 - acc: 0.9997

Epoch 114/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0013 - acc: 0.9995

Epoch 115/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0010 - acc: 0.9996

Epoch 116/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0013 - acc: 0.9996

Epoch 117/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - lo

25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 6.6793e-04 - acc: 0.9998

Epoch 166/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 8.6072e-04 - acc: 0.9998

Epoch 167/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 3.8341e-04 - acc: 0.9999

Epoch 168/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24us/step - loss: 0.0010 - acc: 0.9996

Epoch 169/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 0.0018 - acc: 0.9994

Epoch 170/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 6.1805e-04 - acc: 0.9998

Epoch 171/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 24

25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 5.7077e-04 - acc: 0.9998

Epoch 220/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 5.1520e-04 - acc: 0.9998

Epoch 221/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 3.7409e-04 - acc: 0.9998

Epoch 222/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 3.5369e-04 - acc: 0.9999

Epoch 223/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 26us/step - loss: 2.8971e-04 - acc: 0.9999

Epoch 224/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 3.5611e-04 - acc: 0.9999

Epoch 225/300
25010/25010 [==============================]25010/25010 [==============================]

25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 1.6345e-04 - acc: 1.0000

Epoch 274/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 6.3631e-05 - acc: 1.0000

Epoch 275/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 5.2757e-05 - acc: 1.0000

Epoch 276/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 23us/step - loss: 4.1311e-04 - acc: 0.9998

Epoch 277/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 22us/step - loss: 2.7042e-05 - acc: 1.0000

Epoch 278/300
25010/25010 [==============================]25010/25010 [==============================] - 1s 25us/step - loss: 3.7967e-05 - acc: 1.0000

Epoch 279/300
25010/25010 [==============================]25010/25010 [==============================]

## Test data

In [18]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [19]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [23]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [24]:
test_data_normalized.shape

(1000000, 30)

## Predictions

In [38]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [29]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [39]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(437, 13)

In [40]:
wrong_answers['labels'].value_counts()

4    218
5    168
0     32
7     16
1      2
9      1
Name: labels, dtype: int64

In [41]:
wrong_answers[0:10]

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
34,35,3,13,3,12,2,10,2,1,1,11,0,4
1449,1450,2,2,2,3,2,1,2,11,2,10,0,5
1818,1819,3,1,1,11,3,10,4,12,2,13,0,4
4202,4203,4,10,2,11,1,12,3,13,4,1,0,4
5833,5834,3,8,3,13,3,1,3,10,3,12,0,5
12027,12028,2,13,2,11,2,10,1,12,1,1,0,4
15248,15249,3,2,3,13,3,3,3,4,3,10,0,5
16210,16211,2,10,2,1,2,3,2,12,2,11,0,5
18740,18741,1,11,1,1,3,13,3,10,2,12,0,4
20512,20513,2,11,4,12,1,13,3,1,1,10,0,4


In [42]:
wrong_answers[0:10].drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

34       K♣, Q♣, 10♦, A♦, J♥
1449     2♦, 3♦, A♦, J♦, 10♦
1818     A♣, J♥, 10♣, Q♠, K♦
4202     10♠, J♦, Q♥, K♣, A♠
5833     8♣, K♣, A♣, 10♣, Q♣
12027    K♦, J♦, 10♦, Q♥, A♥
15248    2♣, K♣, 3♣, 4♣, 10♣
16210    10♦, A♦, 3♦, Q♦, J♦
18740    J♥, A♥, K♣, 10♣, Q♦
20512    J♦, Q♠, K♥, A♣, 10♥
dtype: object